# Spotify Roblox API Project

In [1]:
import configparser
import spotipy
from utils.auth_init import auth_init
import utils.spotify_functions as spotify_functions
import utils.roblox_functions as roblox_functions
import utils


Enter the URL you were redirected to:  https://www.google.com/?code=AQDG5VQvPx246HUtr33CxcRWF9DXxx5clIvsXk6vo-Cf7tJmWlPb9SjyvMsmpWh8o370r6XNcHlKVtYTq6mfzQjO8PuBqWZWEniL7ydmeHZZlFqgOGS72YcbQgg2R2wTb8xFZg7dZHpyT8hdJuNsLYV7x7id0NvtESATJ41auNjhStqGA_f7sR7uOAk4cBkXLQDd46ApOJ_wAaDi4OS9kI8k6uzfTjqfiuAPWqtzKzyoRzkLv6fJ0xuQtMdauXQDHZ3Ka8jRX10HQwU7Le8kDUgZefgHSJTKE-PA5PJwnucWQ2q-g4llBgf012GyW4fsiHY6ga_CqF7DNBtmCWWG1uym4Q


In [2]:
# Check key folder and if does not exist create empty templates for keys
auth_init()

auth/spotify_keys.ini already exists.
auth/roblox_keys.ini already exists.


## Code to Create Spotify Playlist Based on User

In [3]:
import robloxpy
import requests
import webbrowser
import spotipy
import spotipy.util as util
import json
import base64
from roblox import Client
import roblox.thumbnails as thumbnail
import asyncio
from utils.roblox_functions import *
import utils.spotify_functions as spotify_functions
import spotipy
import pandas as pd 
from spotipy.oauth2 import SpotifyClientCredentials


In [4]:
# Get Spotify Authentication
keys_path = "auth/spotify_keys.ini"
spotify_auth = spotify_functions.SpotifyAuth(keys_path)

# Authenticate Spotify and get credentials manager as sp
client_credentials_manager = SpotifyClientCredentials(spotify_auth.client_id, spotify_auth.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Spotify scopes (INCLUDE ugc-image-upload)
scope = "user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public playlist-modify-private ugc-image-upload"
token = util.prompt_for_user_token(spotify_auth.username, scope, client_id=spotify_auth.client_id, client_secret=spotify_auth.client_secret, redirect_uri=spotify_auth.redirectURI)

sp = spotipy.Spotify(auth=token)

In [5]:
# Read Roblox Cookie to get specific user
RobloxAuth = RobloxAuthConfig('auth/roblox_keys.ini')
cookie = RobloxAuth.cookie

# Setting Roblox Cookie to Client
client = Client(cookie)

async def main():
    user = await client.get_authenticated_user()
    # Test to see if you are in
    # *** ID CAN BE PRINTED: it is found in Roblox profile URL***
    print("ID:", user.id)
await main()

ID: 7520736841


In [8]:
# Input desired Roblox Username
desired_name = "Funky"
outputted_names = []

async def main():
    # Search the username and X results
    users = client.user_search(desired_name, max_items =10)
    async for user in users:
        # Add to list
        outputted_names.append(user.name)
        outputted_names.append(user.id)
        
await main()

formatted_names = []
for i in range(0, len(outputted_names), 2):
    # Format list to dictonary to access Name & ID
    formatted_names.append({'name': outputted_names[i], 'id': outputted_names[i + 1]})

formatted_names

[{'name': 'funky', 'id': 115158}]

In [9]:
# Pick a user: this could be random but in this case its top result
desired_user = formatted_names[0]
user = await client.get_user(desired_user["id"])
# Get the avatar thumbnaill found on their profile
user_thumbnails = await client.thumbnails.get_user_avatar_thumbnails(
    users=[user],
    type=thumbnail.AvatarThumbnailType.full_body,
    size=(420, 420)
)

# Print the URL of user image
if len(user_thumbnails) > 0:
    user_thumbnail = user_thumbnails[0]
    user_image = user_thumbnail.image_url
    print('Url of user image:')
    print(user_image)

Url of user image:
https://tr.rbxcdn.com/30DAY-Avatar-5F0AB89B2E5D51F8701D55CE31295026-Png/420/420/Avatar/Png/noFilter


In [10]:
# Search Spotify based on the Robloxians Username
track_results = sp.search(q=desired_user["name"], type="track", limit = 10)
# print(track_results)

In [11]:
# function to clean dataframe in spotify_functions.py - takes in track results, clean into a nice dataframe
clean_spotify_df = spotify_functions.spotify_tracks_to_df(track_results)
clean_spotify_df

,song,artists
0,Funky Friday,Dave & Fredo
1,Funkytown,Lipps Inc.
2,Funkytown,Lipps Inc.
3,Thiago Silva,Dave & AJ Tracey
4,Funky Friday,Dave & Fredo
5,Play That Funky Music,Wild Cherry
6,Funkytown - Single Version,Lipps Inc.
7,Funky Cold Medina,Tone-Loc
8,Funky Kingston,Toots & The Maytals
9,Funky Shit,The Prodigy


## Code to Parse Roblox Lua

In [20]:
import glob

# get all lua paths
directory = 'lua_scripts/*'

lua_file_paths = glob.glob(directory)
lua_file_paths


['lua_scripts\\display_script.lua',
 'lua_scripts\\parameter_classes.lua',
 'lua_scripts\\ui_script.lua']

In [21]:
import re
# can move this to script file but leaving it hear for you to see regex

def sub_song_artist_lua(clean_spotify_df, lua_param_file_path):
    """Takes in spotify clean dataframe and lua parameter file,
    return substituted code using regex """

    # Read the Lua script from the file
    file_path = lua_param_file_path
    with open(file_path, 'r') as file:
        lua_script = file.read()

    text = lua_script

    # Regex pattern to match all song entries - artists = and song =
    pattern = r'{artist = "([^"]+)", song = "([^"]+)"}'

    # Find all matches
    matches = list(re.finditer(pattern, text))

    # Substitute each match one by one
    for i,match in enumerate(matches):
        artist, song = match.groups()
        print(f"Matched artist: {artist}, song: {song}")
        
        # Modify text: replace current match with a custom substitution
        text = re.sub(re.escape(match.group(0)), f'{{artist = "{clean_spotify_df['artists'][i]}", song = "{clean_spotify_df['song'][i]}"}}', text, count=1)

    print("\nModified Text:\n")
    print(text)

    return text

def read_lua(file_path):
    # Read the Lua script from the file
    with open(file_path, 'r') as file:
        lua_script = file.read()

    text = lua_script

    return text





In [23]:
# call function to substitute parameters in lua template with new spotify song and artist names
sub_params_lua_str = sub_song_artist_lua(clean_spotify_df,'lua_scripts\\parameter_classes.lua')
display_script_lua = read_lua('lua_scripts\\display_script.lua')
ui_script_lua = read_lua('lua_scripts\\ui_script.lua')

# Add to other scripts to make complete script, \n required to keep lua formatting
complete_display_script_lua = sub_params_lua_str+"\n"+display_script_lua
complete_ui_script_lua = sub_params_lua_str+"\n"+ui_script_lua


Matched artist: Artist 1, song: Song 1
Matched artist: Artist 2, song: Song 2
Matched artist: Artist 3, song: Song 3
Matched artist: Artist 4, song: Song 4
Matched artist: Artist 5, song: Song 5
Matched artist: Artist 6, song: Song 6
Matched artist: Artist 7, song: Song 7
Matched artist: Artist 8, song: Song 8
Matched artist: Artist 9, song: Song 9
Matched artist: Artist 10, song: Song 10

Modified Text:

-- ParameterStart
local data = {
	{artist = "Dave & Fredo", song = "Funky Friday"},
	{artist = "Lipps Inc.", song = "Funkytown"},
	{artist = "Lipps Inc.", song = "Funkytown"},
	{artist = "Dave & AJ Tracey", song = "Thiago Silva"},
	{artist = "Dave & Fredo", song = "Funky Friday"},
	{artist = "Wild Cherry", song = "Play That Funky Music"},
	{artist = "Lipps Inc.", song = "Funkytown - Single Version"},
	{artist = "Tone-Loc", song = "Funky Cold Medina"},
	{artist = "Toots & The Maytals", song = "Funky Kingston"},
	{artist = "The Prodigy", song = "Funky Shit"},
}
-- ParameterEnd


In [15]:
# Get script instance from roblox
main_object_path = ("ServerScriptService/DisplayIterator")
instance_id = find_instance_id(RobloxAuth,object_path=main_object_path)

ui_object_path = ("StarterPlayer/StarterCharacterScripts/UpdateUI")
instance_id_2 = find_instance_id(RobloxAuth,object_path=ui_object_path)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

At level: ServerScriptService


 50%|██████████████████████████████████████████                                          | 1/2 [00:10<00:10, 10.61s/it]

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/108471ac-b6e3-9bdc-0733-9ba500000350', 'hasChildren': True, 'engineInstance': {'Id': '108471ac-b6e3-9bdc-0733-9ba500000350', 'Parent': '701cc964-17d3-e843-073b-ee1000000001', 'Name': 'ServerScriptService', 'Details': {}}}
At level: DisplayIterator


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:16<00:00,  8.01s/it]


...

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/108471ac-b6e3-9bdc-0733-9ba500000325', 'hasChildren': True, 'engineInstance': {'Id': '108471ac-b6e3-9bdc-0733-9ba500000325', 'Parent': '701cc964-17d3-e843-073b-ee1000000001', 'Name': 'StarterPlayer', 'Details': {}}}
At level: StarterCharacterScripts


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:10<00:05,  5.36s/it]

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/108471ac-b6e3-9bdc-0733-9ba50000037b', 'hasChildren': True, 'engineInstance': {'Id': '108471ac-b6e3-9bdc-0733-9ba50000037b', 'Parent': '108471ac-b6e3-9bdc-0733-9ba500000325', 'Name': 'StarterCharacterScripts', 'Details': {}}}
At level: UpdateUI


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.36s/it]

Response: 200
Found dictionary: {'path': 'universes/6737450158/places/106470241725015/instances/6fee2929-cab0-0158-073a-4bad00016972', 'hasChildren': False, 'engineInstance': {'Id': '6fee2929-cab0-0158-073a-4bad00016972', 'Parent': '108471ac-b6e3-9bdc-0733-9ba50000037b', 'Name': 'UpdateUI', 'Details': {'LocalScript': {'Enabled': True, 'Source': '-- ParameterStart\nlocal data = {\n\t{artist = "M.E.R.L.O.T", song = "Ventitre"},\n\t{artist = "Tedi hjk", song = "Veinte"},\n\t{artist = "MatAre", song = "Veinte"},\n\t{artist = "Ventitre & 17WRLDRIOT", song = "Storie scritte"},\n\t{artist = "Ventie20", song = "Outro"},\n\t{artist = "Zain Willis", song = "Ventis Island"},\n\t{artist = "Adrian Peacock & Ora Singers & Suzi Digby", song = "Venite, Gaudete!"},\n\t{artist = "Adrian Peacock & The Chapel Choir of Pembroke College, Cambridge & The Pembroke College Girls’ Choir & Anna Lapwood", song = "Venite, Gaudete!"},\n\t{artist = "Samuel Arnold & The Choir of York Minster & David Pipe & Robert Sha

In [24]:
# Updating script to roblox using wrapper. See wrapper functions in roblox_functions.py

propertyName = "Source" # API defined - not sure if this ever changes
propertyValue = complete_display_script_lua  # The new script content
instanceType = "Script"  # Name of script instance (I think)
update_instance(RobloxAuth,instance_id,propertyName=propertyName,propertyValue=propertyValue,instanceType=instanceType)

propertyName = "Source"
propertyValue = complete_ui_script_lua 
instanceType = "LocalScript" 
update_instance(RobloxAuth,instance_id_2,propertyName=propertyName,propertyValue=propertyValue,instanceType=instanceType)


{'Script': {'Source': '-- ParameterStart\nlocal data = {\n\t{artist = "Dave & Fredo", song = "Funky Friday"},\n\t{artist = "Lipps Inc.", song = "Funkytown"},\n\t{artist = "Lipps Inc.", song = "Funkytown"},\n\t{artist = "Dave & AJ Tracey", song = "Thiago Silva"},\n\t{artist = "Dave & Fredo", song = "Funky Friday"},\n\t{artist = "Wild Cherry", song = "Play That Funky Music"},\n\t{artist = "Lipps Inc.", song = "Funkytown - Single Version"},\n\t{artist = "Tone-Loc", song = "Funky Cold Medina"},\n\t{artist = "Toots & The Maytals", song = "Funky Kingston"},\n\t{artist = "The Prodigy", song = "Funky Shit"},\n}\n-- ParameterEnd\n\nlocal displayGroup = workspace.Displays\nlocal display_no = 0\t\n\nlocal displays = {}\nfor _, display in ipairs(displayGroup:GetChildren()) do\n\tif string.match(display.Name, "SpotifyDisplay%d+") then\n\t\ttable.insert(displays, {\n\t\t\tartist_display = display["Artist Display"].SurfaceGui.TextLabel,\n\t\t\tsong_display = display["Song Display"].SurfaceGui["Song T

## Create Spotify Playlist 
(doing it here so that it will launch the same time as the fade in for the game)

In [17]:
song_data = track_results["tracks"]["items"]

# Get song URIs into a list
song_uris = []
for song in song_data:
    song_uris.append(song["uri"])

my_playlist = sp.user_playlist_create(user=spotify_auth.username, name=desired_user["name"], public=True,
                                      description=f"Playlist for the Robloxian {desired_user["name"]}!")
# Add the songs
results = sp.user_playlist_add_tracks(spotify_auth.username, my_playlist['id'], song_uris)
print(results)

# Convert the user thumbnail through base64 enconde
def get_as_base64(url):
    response = requests.get(url)
    return base64.b64encode(response.content).decode('utf-8')

get_as_base64(user_image)

# Get image and upload the image as playlist cover
user_image_base64 = get_as_base64(user_image)
sp.playlist_upload_cover_image(my_playlist['id'], user_image_base64)

# Open Spotify
webbrowser.open(my_playlist['external_urls']['spotify'])

{'snapshot_id': 'AAAAAkZJ/1sbVtuMcLCtu+2+LEsESjuI'}


True

## For Future Reference

This is the code to publish game automatically. Currently doesn't work. Based on V1 version of Roblox API.

```python
api_keyHeaderKey = "x-api-key"
contentTypeHeaderKey = "Content-type"
contentTypeHeaderValue = "application/json"
data_file_path = "/path/to/your/file.rbxl"

def publish_experience(RobloxAuth):
    url = f"https://apis.roblox.com/universes/v1/{RobloxAuth.universe_id}/places/{RobloxAuth.place_id}/versions"
    headerData = {api_keyHeaderKey: RobloxAuth.api_key,
        contentTypeHeaderKey: contentTypeHeaderValue}
    data = {"versionType": "Published"}

    try:
        response = requests.post(url, headers=headerData, json=data) 
        response.raise_for_status()
        print(f"Publish successful: {response.json()}")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error publishing: {e}")
        if response: 
            print(f"Roblox error response: {response.text}")
        return None

publish_response = publish_experience(RobloxAuth)
````